In [11]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import lightgbm as lgb
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn.preprocessing import MinMaxScaler

pd.options.display.max_columns = 500
#import pandas_profiling

In [2]:
# ## plotly 
# import plotly.plotly as py
# import plotly.graph_objs as go
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# init_notebook_mode(connected=True)

In [3]:
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:50].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    #plt.savefig('lgbm_importances01.png')

In [4]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

## FE

In [5]:
train_df.tail(10)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
199990,train_199990,1,14.1475,1.8568,11.0066,3.6779,12.1944,-16.5936,5.3217,14.8508,3.3377,6.1650,6.1414,0.0827,14.0400,1.6150,7.5366,14.4789,6.2584,2.1186,26.7359,0.5702,4.0029,15.5389,5.4881,2.4573,18.2939,13.8830,-16.1930,-1.8986,4.8970,7.5907,0.8798,8.5026,4.3286,12.8019,11.9591,2.9620,-4.4218,5.8075,6.2756,-1.8136,-24.1308,24.6378,11.9017,11.6704,11.5972,35.0815,9.5150,-5.1348,37.5782,13.2321,12.6700,29.2393,-6.1634,6.5313,0.4095,7.6697,17.3582,4.8764,-5.3656,7.7960,8.2764,-22.6021,1.6359,-5.1907,6.5715,1.6882,6.2847,3.3086,5.0167,-7.6718,23.4462,0.7742,1.1682,31.0228,32.3939,16.7054,10.3803,15.6047,4.7141,13.5369,-2.3319,11.8592,0.9950,17.4239,-0.8730,13.8346,10.0802,10.3809,7.0715,12.7042,-25.1787,7.1149,12.2700,10.9310,7.7426,0.4302,13.6463,38.9630,2.3068,-0.3915,-8.5204,20.1458,19.0598,1.4362,11.6263,4.1638,7.5614,19.4758,14.2716,16.1327,3.1721,9.3863,4.1246,4.7095,2.6171,4.9913,2.1080,31.6905,-1.7796,0.4008,22.8108,9.0540,0.5427,-6.9109,6.0711,12.7660,12.7326,3.8427,1.2784,19.1283,12.6645,0.4973,8.1403,6.4378,-8.2097,1.5164,34.2795,14.7229,5.3334,-9.0510,-3.4490,-6.8524,17.0059,11.4479,7.6751,3.8062,9.9805,-11.8864,3.8402,21.7955,17.6339,6.8544,10.9870,16.7374,4.9328,-8.8372,15.2210,-9.6098,25.5304,7.5533,12.4722,5.7143,4.3119,11.5818,-1.0976,15.9085,3.3110,-0.8404,5.9175,4.7656,-7.7289,6.2246,26.4941,3.7733,31.7617,8.5447,-2.1253,9.6392,-2.7775,-2.2930,-0.4321,8.2347,-16.8808,6.8203,15.0222,3.3871,9.8074,-36.3302,16.0983,0.8156,-6.4708,4.7287,1.9034,7.2324,20.6047,1.7170,-4.0032,9.1627,13.8077,-1.9646
199991,train_199991,0,9.9909,2.5523,11.9653,6.3958,13.5497,-9.5293,6.0864,14.1789,-2.1245,9.8862,-3.1886,-0.2491,13.9248,13.8947,9.6829,13.9033,10.2258,5.9237,17.3947,1.9836,9.2717,28.8883,5.6900,3.1928,8.9120,13.7650,-11.6627,-1.4927,5.4689,1.3060,2.4111,12.4307,3.1743,14.9818,11.8855,1.8232,9.2384,5.5283,9.1446,5.1439,-3.5793,1.0812,10.7423,11.6788,1.9277,-19.9550,6.4090,-23.0383,-5.0783,16.4067,13.4490,3.3566,7.4365,5.1999,4.9993,6.6391,15.4445,6.4535,2.7636,9.3584,22.6203,1.7745,1.2965,-1.7242,5.1655,-3.2149,6.5610,16.0365,5.0258,-6.1367,48.5375,0.8261,3.0558,16.2346,39.7911,19.4477,10.8329,15.5222,4.4408,13.5912,1.9444,18.0211,-23.2330,5.7972,-11.0950,13.2983,4.5397,7.8958,3.0405,-4.4533,-25.4294,6.8083,21.1329,10.9765,11.9918,0.3610,21.2432,-2.4605,2.3231,-3.0889,2.6465,2.2822,28.6410,1.8103,12.2662,4.9371,9.1332,5.7253,14.1759,14.4442,9.2293,7.8426,1.8774,8.9777,1.8659,4.1717,4.7503,35.5995,-10.8612,5.7606,22.9226,12.3712,4.2567,9.2771

In [6]:
train_len = train_df.shape[0]
merged_df = pd.concat([train_df, test_df],sort=False)


var_list = [f for f in merged_df.columns if 'var' in f]
new_var_list = [f+'_rank' for f in var_list]

In [8]:
#merged_df[new_var_list] = merged_df[var_list].apply(lambda x: pd.qcut(x, q=2, labels=False))

scaler = MinMaxScaler()

merged_df[var_list] = scaler.fit_transform(merged_df[var_list])

In [17]:
kmeans = MiniBatchKMeans(n_clusters=20, random_state=0,batch_size=40000).fit(merged_df[var_list])

In [18]:
merged_df['cluster'] = kmeans.predict(merged_df[var_list])

In [19]:
merged_df.groupby('cluster')['target'].mean()

cluster
0     0.170412
1     0.086861
2     0.076114
3     0.110917
4     0.109108
5     0.110998
6     0.131395
7     0.123753
8     0.080734
9     0.056906
10    0.080937
11    0.071917
12    0.119617
13    0.112254
14    0.072821
15    0.089947
16    0.100436
17    0.084721
18    0.152088
19    0.077937
Name: target, dtype: float64

In [20]:


#merged_df[new_var_list] = merged_df[var_list].rank(pct=True)

train_df = merged_df.iloc[:train_len]
test_df = merged_df.iloc[train_len:]

# del merged_df
# gc.collect()

In [21]:
# #merged_df['target'] = np.where(merged_df.target.isna(),3,merged_df.target)
# merged_df.groupby('target')[var_list].mean()

In [22]:
test_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,cluster
0,test_0,NaN,0.491396,0.897837,0.628717,0.715833,0.548256,0.605868,0.583008,0.563036,0.611866,0.674016,0.481443,0.502258,0.433479,0.224288,0.481552,0.510767,0.418937,0.271997,0.506947,0.599931,0.243708,0.488649,0.347060,0.486421,0.274367,0.723250,0.374172,0.416760,0.540640,0.478163,0.654120,0.421748,0.693525,0.401839,0.443413,0.638855,0.421076,0.357357,0.445720,0.780374,0.367258,0.777621,0.526482,0.616395,0.528090,0.382464,0.546175,0.825746,0.533022,0.497323,0.557644,0.528447,0.473185,0.813736,0.465730,0.607985,0.628672,0.517797,0.263193,0.551762,0.514066,0.595841,0.397306,0.565242,0.562894,0.585465,0.541750,0.606453,0.352166,0.598889,0.618069,0.571190,0.280072,0.691240,0.525939,0.586475,0.516169,0.282698,0.368759,0.897291,0.567416,0.587951,0.858610,0.662618,0.557678,0.519196,0.447038,0.690740,0.733314,0.683953,0.607569,0.573295,0.551588,0.408143,0.191764,0.425572,0.328271,0.662231,0.527133,0.741335,0.480092,0.602447,0.648979,0.412121,0.577396,0.424199,0.511187,0.348748,0.284615,0.368999,0.455206,0.505969,0.629332,0.716433,0.322900,0.447496,0.565736,0.367361,0.497733,0.716841,0.657945,0.464562,0.464881,0.543956,0.363901,0.462238,0.672580,0.663211,0.339046,0.499085,0.551106,0.595882,0.797711,0.730694,0.552071,0.585268,0.317316,0.508499,0.286484,0.317332,0.281695,0.358320,0.673713,0.516394,0.594301,0.457164,0.562317,0.351048,0.520003,0.518613,0.588630,0.510624,0.489165,0.311073,0.864667,0.598706,0.294747,0.535986,0.407999,0.502555,0.202807,0.374690,0.369151,0.359852,0.383653,0.556347,0.307398,0.631153,0.478060,0.327433,0.612012,0.598001,0.625209,0.743509,0.453933,0.521158,0.581971,0.336894,0.401178,0.462912,0.485772,0.557511,0.219720,0.304281,0.792989,0.601959,0.497131,0.339062,0.355245,0.639518,0.346499,0.749230,0.195180,0.473444,0.260567,0.788047,0.570992,0.753642,0.453227,0.452307,5
1,test_1,NaN,0.376861,0.641132,0.533050,0.395056,0.355501,0.573124,0.610338,0.578906,0.294934,0.279021,0.496457,0.595236,0.547435,0.302647,0.338429,0.397789,0.380199,0.724078,0.762831,0.265738,0.468676,0.223696,0.531220,0.317972,0.647038,0.404373,0.480016,0.262316,0.540237,0.434109,0.482189,0.539933,0.503279,0.493471,0.244831,0.454114,0.696395,0.267118,0.531100,0.434984,0.848554,0.331584,0.372012,0.580295,0.274558,0.322050,0.613609,0.443207,0.396724,0.411676,0.290976,0.725559,0.626320,0.716899,0.630018,0.601654,0.719960,0.608715,0.457127,0.585130,0.272526,0.396471,0.546406,0.455701,0.450305,0.592364,0.361420,0.582139,0.693032,0.360985,0.359644,0.514060,0.421680,0

## Model

In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

n_folds = 5
random_seed = 26
model = 'kmeans_20K'


model_name = "{0}_{1}_folds".format(model, n_folds)
print("Model: {}".format(model_name))

Model: kmeans_20K_5_folds


In [24]:
#feats = train_df.iloc[:,2:].columns.tolist();feats

#exclusion = ['ID_code', 'target']+ rank_var_list
exclusion = ['ID_code', 'target'] 

feats = [c for c in train_df.columns if c not in exclusion]

#cat_feats = [c for c in feats if c not in var_list]

In [25]:
feats

['var_0',
 'var_1',
 'var_2',
 'var_3',
 'var_4',
 'var_5',
 'var_6',
 'var_7',
 'var_8',
 'var_9',
 'var_10',
 'var_11',
 'var_12',
 'var_13',
 'var_14',
 'var_15',
 'var_16',
 'var_17',
 'var_18',
 'var_19',
 'var_20',
 'var_21',
 'var_22',
 'var_23',
 'var_24',
 'var_25',
 'var_26',
 'var_27',
 'var_28',
 'var_29',
 'var_30',
 'var_31',
 'var_32',
 'var_33',
 'var_34',
 'var_35',
 'var_36',
 'var_37',
 'var_38',
 'var_39',
 'var_40',
 'var_41',
 'var_42',
 'var_43',
 'var_44',
 'var_45',
 'var_46',
 'var_47',
 'var_48',
 'var_49',
 'var_50',
 'var_51',
 'var_52',
 'var_53',
 'var_54',
 'var_55',
 'var_56',
 'var_57',
 'var_58',
 'var_59',
 'var_60',
 'var_61',
 'var_62',
 'var_63',
 'var_64',
 'var_65',
 'var_66',
 'var_67',
 'var_68',
 'var_69',
 'var_70',
 'var_71',
 'var_72',
 'var_73',
 'var_74',
 'var_75',
 'var_76',
 'var_77',
 'var_78',
 'var_79',
 'var_80',
 'var_81',
 'var_82',
 'var_83',
 'var_84',
 'var_85',
 'var_86',
 'var_87',
 'var_88',
 'var_89',
 'var_90',
 'var_91'

In [ ]:
clfs = []
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
oof_preds = np.zeros((len(train_df), 1))
test_preds = np.zeros((len(test_df), 1))


X = train_df[feats]
y = train_df['target']
X_test = test_df[feats]
test_ids = test_df.ID_code.values


parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    #'scale_pos_weight': 400,
    #'device' : 'gpu' ,
    'boosting': 'gbdt',
    'num_leaves': 31, #31
    'feature_fraction': 0.5,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'learning_rate': 0.05, #0.05
    'verbose': 30
    #'min_data_in_leaf': 200
}

feature_importance_df = pd.DataFrame()
for fold_, (trn_, val_) in enumerate(folds.split(X, y)):
    print("Current Fold: {}".format(fold_+1))
    trn_x, trn_y = X.iloc[trn_, :], y[trn_]
    val_x, val_y = X.iloc[val_, :], y[val_]


    trn_lgb = lgb.Dataset(trn_x, trn_y)
    val_lgb = lgb.Dataset(val_x, val_y)
    clf = lgb.train(parameters,
                     train_set=trn_lgb,
                     #valid_sets=[valid_data_lgb,holdout_data_lgb],
                     valid_sets=[trn_lgb, val_lgb],
                     num_boost_round=3000,
                     early_stopping_rounds=50,
                     verbose_eval=False)
    


    val_pred = clf.predict(val_x)
    test_fold_pred = clf.predict(X_test)

    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    oof_preds[val_, :] = val_pred.reshape((-1, 1))
    test_preds += test_fold_pred.reshape((-1, 1))
    
   # print('getting feature importance')
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    
test_preds /= n_folds
roc_score = roc_auc_score(y, oof_preds.ravel())
print("Overall AUC = {}".format(roc_score))


print("Saving submission file")
sample = pd.read_csv('../data/sample_submission.csv')
sample.target = test_preds.astype(float)
sample.ID_code = test_ids
sample.to_csv('../submissions/{}_{}.csv'.format(model_name,str(roc_score)), index=False)

display_importances(feature_importance_df)


Current Fold: 1
AUC = 0.8920333921399367
Current Fold: 2
AUC = 0.8903448835143922
Current Fold: 3
AUC = 0.8939476533397492
Current Fold: 4


In [74]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [75]:


shutil.copyfile(os.path.basename(NOTEBOOK_FULL_PATH), 
                             '../models/{}_{}.ipynb'.format(model_name, str(roc_score)))


'../models/rank_pct_1_0_5_folds_0.8930529374503305.ipynb'

## Post-model evaluation

In [88]:
y.shape

(200000,)

In [90]:
oof_preds

array([[0.02178814],
       [0.75473365],
       [0.02379914],
       ...,
       [0.33932852],
       [0.25191656],
       [0.05398939]])

In [91]:
train_df['pred'] = oof_preds

C:\Users\TanQ\AppData\Local\Continuum\anaconda3\envs\work\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [92]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,...,var_76_rank,var_77_rank,var_78_rank,var_79_rank,var_80_rank,var_81_rank,var_82_rank,var_83_rank,var_84_rank,var_85_rank,var_86_rank,var_87_rank,var_88_rank,var_89_rank,var_90_rank,var_91_rank,var_92_rank,var_93_rank,var_94_rank,var_95_rank,var_96_rank,var_97_rank,var_98_rank,var_99_rank,var_100_rank,var_101_rank,var_102_rank,var_103_rank,var_104_rank,var_105_rank,var_106_rank,var_107_rank,var_108_rank,var_109_rank,var_110_rank,var_111_rank,var_112_rank,var_113_rank,var_114_rank,var_115_rank,var_116_rank,var_117_rank,var_118_rank,var_119_rank,var_120_rank,var_121_rank,var_122_rank,var_123_rank,var_124_rank,var_125_rank,var_126_rank,var_127_rank,var_128_rank,var_129_rank,var_130_rank,var_131_rank,var_132_rank,var_133_rank,var_134_rank,var_135_rank,var_136_rank,var_137_rank,var_138_rank,var_139_rank,var_140_rank,var_141_rank,var_142_rank,var_143_rank,var_144_rank,var_145_rank,var_146_rank,var_147_rank,var_148_rank,var_149_rank,var_150_rank,var_151_rank,var_152_rank,var_153_rank,var_154_rank,var_155_rank,var_156_rank,var_157_rank,var_158_rank,var_159_rank,var_160_rank,var_161_rank,var_162_rank,var_163_rank,var_164_rank,var_165_rank,var_166_rank,var_167_rank,var_168_rank,var_169_rank,var_170_rank,var_171_rank,var_172_rank,var_173_rank,var_174_rank,var_175_rank,var_176_rank,var_177_rank,var_178_rank,var_179_rank,var_180_rank,var_181_rank,var_182_rank,var_183_rank,var_184_rank,var_185_rank,var_186_rank,var_187_rank,var_188_rank,var_189_rank,var_190_rank,var_191_rank,var_192_rank,var_193_rank,var_194_rank,var_195_rank,var_196_rank,var_197_rank,var_198_rank,var_199_rank,pred
0,train_0,0.0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,4.6667,3.8743,-5.2387,7.3746,11.5767,12.0446,11.6418,-7.0170,5.9226,-14.2136,16.0283,5.3253,12.9194,29.0460,-0.6940,5.1736,-0.7474,14.8322,11.2668,5.3822,2.0183,10.1166,16.1828,4.9590,2.0771,-0.2154,8.6748,9.5319,5.8056,22.4321,5.0109,-4.7010,21.6374,0.5663,5.1999,8.8600,43.1127,18.3816,-2.3440,23.4104,6.5199,12.1983,13.6468,13.8372,1.3675,2.9423,-4.5213,21.4669,9.3225,16.4597,7.9984,-1.7069,-21.4494,6.7806,11.0924,9.9913,14.8421,0.1812,8.9642,16.2572,2.1743,-3.4132,9.4763,13.3102,26.5376,1.4403,14.7100,6.0454,9.5426,17.1554,14.1104,24.3627,2.0323,6.7602,3.9141,-0.4851,2.5240,1.5093,2.5516,15.5752,-13.4221,7.2739,16.0094,9.7268,0.8897,...,0.149740,0.851252,0.687106,0.035019,0.843741,0.332097,0.680155,0.580236,0.387289,0.756319,0.664846,0.801168,0.406734,0.057969,0.373588,0.091916,0.364665,0.150326,0.914253,0.670306,0.292324,0.439945,0.709815,0.073491,0.966692,0.496190,0.670069,0.270106,0.945436,0.978002,0.682996,0.479762,0.270314,0.898384,0.188282,0.643261,0.606224,0.024567,0.293305,0.399529,0.521334,0.491107,0.190424,0.832815,0.257181,0.196555,0.478909,0.160370,0.486260,0.056399,0.765304,0.326871,0.416558,0.503275,0.731633,0.351265,0.890018,0.269597,0.952459,0.830864,0.702192,0.934665,0.6317

In [101]:
most_wrong_0 = train_df.loc[train_df.target==0].sort_values('pred',ascending=False).head(100)
most_wrong_1 = train_df.loc[train_df.target==1].sort_values('pred',ascending=True).head(100)

In [107]:
most_wrong_0.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_0_rank,var_1_rank,var_2_rank,var_3_rank,var_4_rank,var_5_rank,var_6_rank,var_7_rank,var_8_rank,var_9_rank,var_10_rank,var_11_rank,var_12_rank,var_13_rank,var_14_rank,var_15_rank,var_16_rank,var_17_rank,var_18_rank,var_19_rank,var_20_rank,var_21_rank,var_22_rank,var_23_rank,var_24_rank,var_25_rank,var_26_rank,var_27_rank,var_28_rank,var_29_rank,var_30_rank,var_31_rank,var_32_rank,var_33_rank,var_34_rank,var_35_rank,var_36_rank,var_37_rank,var_38_rank,var_39_rank,var_40_rank,var_41_rank,var_42_rank,var_43_rank,var_44_rank,var_45_rank,var_46_rank,var_47_rank,var_48_rank,var_49_rank,var_50_rank,var_51_rank,var_52_rank,var_53_rank,var_54_rank,var_55_rank,var_56_rank,var_57_rank,var_58_rank,var_59_rank,var_60_rank,var_61_rank,var_62_rank,var_63_rank,var_64_rank,var_65_rank,var_66_rank,var_67_rank,var_68_rank,var_69_rank,var_70_rank,var_71_rank,var_72_rank,var_73_rank,var_74_rank,var_75_rank,var_76_rank,var_77_rank,var_78_rank,var_79_rank,var_80_rank,var_81_rank,var_82_rank,var_83_rank,var_84_rank,var_85_rank,var_86_rank,var_87_rank,var_88_rank,var_89_rank,var_90_rank,var_91_rank,var_92_rank,var_93_rank,var_94_rank,var_95_rank,var_96_rank,var_97_rank,var_98_rank,var_99_rank,var_100_rank,var_101_rank,var_102_rank,var_103_rank,var_104_rank,var_105_rank,var_106_rank,var_107_rank,var_108_rank,var_109_rank,var_110_rank,var_111_rank,var_112_rank,var_113_rank,var_114_rank,var_115_rank,var_116_rank,var_117_rank,var_118_rank,var_119_rank,var_120_rank,var_121_rank,var_122_rank,var_123_rank,var_124_rank,var_125_rank,var_126_rank,var_127_rank,var_128_rank,var_129_rank,var_130_rank,var_131_rank,var_132_rank,var_133_rank,var_134_rank,var_135_rank,var_136_rank,var_137_rank,var_138_rank,var_139_rank,var_140_rank,var_141_rank,var_142_rank,var_143_rank,var_144_rank,var_145_rank,var_146_rank,var_147_rank,var_148_rank,var_149_rank,var_150_rank,var_151_rank,var_152_rank,var_153_rank,var_154_rank,var_155_rank,var_156_rank,var_157_rank,var_158_rank,var_159_rank,var_160_rank,var_161_rank,var_162_rank,var_163_rank,var_164_rank,var_165_rank,var_166_rank,var_167_rank,var_168_rank,var_169_rank,var_170_rank,var_171_rank,var_172_rank,var_173_rank,var_174_rank,var_175_rank,var_176_rank,var_177_rank,var_178_rank,var_179_rank,var_180_rank,var_181_rank,var_182_rank,var_183_rank,var_184_rank,var_185_rank,var_186_rank,var_187_rank,var_188_rank,var_189_rank,var_190_rank,var_191_rank,var_192_rank,var_193_rank,var_194_rank,var_195_rank,var_196_rank,var_197_rank,var_198_rank,var_199_rank,pred

In [109]:
merged_df.groupby('target').median()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_0_rank,var_1_rank,var_2_rank,var_3_rank,var_4_rank,var_5_rank,var_6_rank,var_7_rank,var_8_rank,var_9_rank,var_10_rank,var_11_rank,var_12_rank,var_13_rank,var_14_rank,var_15_rank,var_16_rank,var_17_rank,var_18_rank,var_19_rank,var_20_rank,var_21_rank,var_22_rank,var_23_rank,var_24_rank,var_25_rank,var_26_rank,var_27_rank,var_28_rank,var_29_rank,var_30_rank,var_31_rank,var_32_rank,var_33_rank,var_34_rank,var_35_rank,var_36_rank,var_37_rank,var_38_rank,var_39_rank,var_40_rank,var_41_rank,var_42_rank,var_43_rank,var_44_rank,var_45_rank,var_46_rank,var_47_rank,var_48_rank,var_49_rank,var_50_rank,var_51_rank,var_52_rank,var_53_rank,var_54_rank,var_55_rank,var_56_rank,var_57_rank,var_58_rank,var_59_rank,var_60_rank,var_61_rank,var_62_rank,var_63_rank,var_64_rank,var_65_rank,var_66_rank,var_67_rank,var_68_rank,var_69_rank,var_70_rank,var_71_rank,var_72_rank,var_73_rank,var_74_rank,var_75_rank,var_76_rank,var_77_rank,var_78_rank,var_79_rank,var_80_rank,var_81_rank,var_82_rank,var_83_rank,var_84_rank,var_85_rank,var_86_rank,var_87_rank,var_88_rank,var_89_rank,var_90_rank,var_91_rank,var_92_rank,var_93_rank,var_94_rank,var_95_rank,var_96_rank,var_97_rank,var_98_rank,var_99_rank,var_100_rank,var_101_rank,var_102_rank,var_103_rank,var_104_rank,var_105_rank,var_106_rank,var_107_rank,var_108_rank,var_109_rank,var_110_rank,var_111_rank,var_112_rank,var_113_rank,var_114_rank,var_115_rank,var_116_rank,var_117_rank,var_118_rank,var_119_rank,var_120_rank,var_121_rank,var_122_rank,var_123_rank,var_124_rank,var_125_rank,var_126_rank,var_127_rank,var_128_rank,var_129_rank,var_130_rank,var_131_rank,var_132_rank,var_133_rank,var_134_rank,var_135_rank,var_136_rank,var_137_rank,var_138_rank,var_139_rank,var_140_rank,var_141_rank,var_142_rank,var_143_rank,var_144_rank,var_145_rank,var_146_rank,var_147_rank,var_148_rank,var_149_rank,var_150_rank,var_151_rank,var_152_rank,var_153_rank,var_154_rank,var_155_rank,var_156_rank,var_157_rank,var_158_rank,var_159_rank,var_160_rank,var_161_rank,var_162_rank,var_163_rank,var_164_rank,var_165_rank,var_166_rank,var_167_rank,var_168_rank,var_169_rank,var_170_rank,var_171_rank,var_172_rank,var_173_rank,var_174_rank,var_175_rank,var_176_rank,var_177_rank,var_178_rank,var_179_rank,var_180_rank,var_181_rank,var_182_rank,var_183_rank,var_184_rank,var_185_rank,var_186_rank,var_187_rank,var_188_rank,var_189_rank,var_190_rank,var_191_rank,var_192_rank,var_193_rank,var_194_rank,var_195_rank,var_196_rank,var_197_rank,var_198_rank,var_199_rank
target,,,,,,,,,,,,,